<a href="https://colab.research.google.com/github/dicad/parallel_programming_CuTonala_1_23/blob/BFS%2C-DFS%2C-MPI-and-CUDA/BFS_and_DFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BFS and DFS with MPI and CUDA**

In [2]:
!pip install mpi4py
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2023.1-cp310-cp310-linux_x86_64.whl size=661263 sha256=83811be4af132de3dcb25d12317ee25cb8096b34ea4fd2524f4641e6b51f096e
  Stored in directory: /root/.cache/pip/wheels/46/65/06/b997165edd2fd9690c3497ca54ea4485b571d7bd959c21c6c4
Successfully built pycuda


In [27]:
import numpy as np
from mpi4py import MPI
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
import pycuda.compiler

cuda_code_bfs = """
__global__ void bfs_kernel(int *graph, int *queue, int *visited, int *result, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    while (tid < size) {
        int current_node = queue[tid];
        if (!visited[current_node]) {
            visited[current_node] = 1;
            result[current_node] = 1;

            for (int neighbor = 0; neighbor < size; ++neighbor) {
                if (graph[current_node * size + neighbor] && !visited[neighbor]) {
                    queue[size + tid] = neighbor;
                }
            }
        }

        tid += blockDim.x * gridDim.x;
    }
}
"""

cuda_code_dfs = """
__global__ void dfs_kernel(int *graph, int *stack, int *visited, int *result, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    while (tid < size) {
        int current_node = stack[tid];
        if (!visited[current_node]) {
            visited[current_node] = 1;
            result[current_node] = 1;

            for (int neighbor = 0; neighbor < size; ++neighbor) {
                if (graph[current_node * size + neighbor] && !visited[neighbor]) {
                    stack[size + tid] = neighbor;
                }
            }
        }

        tid += blockDim.x * gridDim.x;
    }
}
"""

# Initialize CUDA context
cuda.init()
dev = cuda.Device(0)  # Adjust the device index as needed
ctx = dev.make_context()

try:
    # Compile CUDA code
    cuda_module_bfs = pycuda.compiler.SourceModule(cuda_code_bfs)
    cuda_module_dfs = pycuda.compiler.SourceModule(cuda_code_dfs)

    # Get CUDA kernel functions
    bfs_kernel = cuda_module_bfs.get_function("bfs_kernel")
    dfs_kernel = cuda_module_dfs.get_function("dfs_kernel")

    def partition_graph_gpu(graph, num_partitions):
        partitions = [np.zeros((len(graph),), dtype=np.int32) for _ in range(num_partitions)]

        for i, node_key in enumerate(graph.keys()):
            partition_id = i % num_partitions
            partitions[partition_id][i] = 1  # Represent the presence of the node in the partition

        return partitions

    def bfs_gpu(graph, start_node, result_gpu):
        visited_gpu = gpuarray.zeros(len(graph), dtype=np.int32)
        queue_gpu = gpuarray.to_gpu(np.array([start_node], dtype=np.int32))

        block_size = 128
        grid_size = (len(graph) + block_size - 1) // block_size

        bfs_kernel(graph, queue_gpu, visited_gpu, result_gpu, np.int32(len(graph)),
                   block=(block_size, 1, 1), grid=(grid_size, 1))

    def dfs_gpu(graph, start_node, result_gpu):
        visited_gpu = gpuarray.zeros(len(graph), dtype=np.int32)
        stack_gpu = gpuarray.to_gpu(np.array([start_node], dtype=np.int32))

        block_size = 128
        grid_size = (len(graph) + block_size - 1) // block_size

        dfs_kernel(graph, stack_gpu, visited_gpu, result_gpu, np.int32(len(graph)),
                   block=(block_size, 1, 1), grid=(grid_size, 1))

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    graph = {
        0: [1, 2],
        1: [0, 3, 4],
        2: [0, 5],
        3: [1, 5],
        4: [1, 5],
        5: [2, 3, 4]
    }

    # Divide the graph among MPI processes
    graph_partitions = partition_graph_gpu(graph, size)
    local_partition = graph_partitions[rank]

    # Perform BFS or DFS on each partition using GPU
    result_gpu = gpuarray.zeros(len(graph), dtype=np.int32)

    if rank == 0:
        # Root process handles overall coordination and final result
        partial_results_gpu = [gpuarray.to_gpu(np.array(local_partition, dtype=np.int32))]
        for i in range(1, size):
            partial_results_gpu.append(comm.recv(source=i))

        for partial_result_gpu in partial_results_gpu:
            bfs_gpu(partial_result_gpu, 0, result_gpu)

        result_cpu = result_gpu.get()
        print("Final result:", set(np.where(result_cpu == 1)[0]))
    else:
        # Worker processes perform BFS or DFS on their respective partitions using GPU
        bfs_gpu(local_partition, rank, result_gpu)
        comm.send(result_gpu, dest=0)

finally:
    # Ensure to clean up the CUDA context
    ctx.pop()


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


Final result: {0}
